In [ ]:
import pandas as pd
import xgboost as xgb
import time
from geopy.geocoders import Nominatim

# 1. Modeli Yükle
print("🧠 Model yükleniyor...")
model = xgb.XGBRegressor()
try:
    model.load_model("ev_fiyat_modeli.json")
    print("✅ Model hazır!")
except Exception as e:
    print(f"❌ HATA: Model dosyası bulunamadı veya bozuk. Hata detayı: {e}")
    # Jupyter'de exit() yerine raise kullanırız veya işlemi durdururuz
    raise SystemExit("Model yüklenemediği için işlem durduruldu.")

# 2. Harita Servisini Başlat
# user_agent kısmına rastgele ama benzersiz bir isim veriyoruz ki servis bizi engellemesin
geolocator = Nominatim(user_agent="batu_ev_fiyat_v1")

print("-" * 60)
print("Lütfen evin özelliklerini girin:")

# --- ADRES GİRİŞİ ---
adres = input("📍 Evin Adresi (Örn: Carmelite Road, Harrow): ")
print("   🔎 Adres haritada aranıyor...")

try:
    # timeout=10 ekledik, internet yavaşsa hemen hata vermesin diye
    location = geolocator.geocode(adres, timeout=10)
    
    if location is None:
        print("\n❌ ADRES BULUNAMADI!")
        print("👉 İpucu: 'Middlesex' gibi eski bölge isimlerini çıkarıp sadece 'Posta Kodu' deneyin.")
        print("👉 Örnek: 'HA3 5NE' veya 'Carmelite Road, Harrow'")
    
    else:
        # Adres bulunduysa işlemlere devam et
        print(f"   ✅ Adres Bulundu: {location.address}")
        print(f"   🌐 Koordinatlar: Enlem {location.latitude:.4f}, Boylam {location.longitude:.4f}")
        print("-" * 30)

        # Koordinatları al
        enlem = location.latitude
        boylam = location.longitude

        # Diğer Özellikleri İste
        try:
            oda_sayisi = int(input("🛏️  Yatak Odası Sayısı: "))
            banyo_sayisi = int(input("🛁  Banyo Sayısı: "))
            metrekare = float(input("📏  Evin Büyüklüğü (m2): "))
            
            print("\n🏠 Ev Tipi Seçiniz:")
            print("   1: Daire (Flat)")
            print("   2: Müstakil (Detached)")
            print("   3: Sıralı Ev (Terraced)")
            ev_tipi_giris = input("   Seçiminiz (1-3): ")
            ev_tipi = int(ev_tipi_giris) if ev_tipi_giris.isdigit() else 0

            # Veriyi Modele Hazırla
            yeni_ev = pd.DataFrame({
                'bedrooms': [oda_sayisi],
                'bathrooms': [banyo_sayisi],
                'floorAreaSqM': [metrekare],
                'latitude': [enlem],
                'longitude': [boylam],
                'propertyType': [ev_tipi], 
                'tenure': [1],             
                'currentEnergyRating': [2] 
            })

            # Tahmin Yap
            print("\n🤖 Yapay zeka fiyatı hesaplıyor...")
            time.sleep(1)
            
            tahmin = model.predict(yeni_ev)[0]

            print("\n" + "="*50)
            print(f"📍 Konum: {location.address[:60]}...")
            print(f"💰 TAHMİNİ FİYAT: £{tahmin:,.2f}")
            print("="*50 + "\n")

        except ValueError:
            print("❌ HATA: Lütfen oda sayısı vb. alanlara sadece SAYI girin.")

except Exception as e:
    print(f"\n❌ Bağlantı Hatası veya Beklenmeyen Durum: {e}")

🧠 Model yükleniyor...
✅ Model hazır!
------------------------------------------------------------
Lütfen evin özelliklerini girin:
   🔎 Adres haritada aranıyor...
❌ Adres bulunamadı! Lütfen daha açık bir adres girin veya posta kodu deneyin.
------------------------------


: 